In [ ]:
#r "nuget:CardanoSharp.Wallet,0.2.0"

Installed Packages cardanosharp.cbor, 0.1.0 cardanosharp.chaos, 0.1.0 CardanoSharp.Wallet, 0.2.0 microsoft.aspnetcore.cryptography.internal, 5.0.6 microsoft.aspnetcore.cryptography.keyderivation, 5.0.6 newtonsoft.json, 13.0.1 petero.numbers, 1.7.4 petero.uriutility, 1.0.0 saucecontrol.blake2fast, 2.0.0

# Wallet Basic Overview

In this section, we're going to review how to:
 - Generate a New Mnemonic
 - Restore a Mnemonic
 - Get the Root Key for the Mnemonic
 - Derive Private and Public Keys
 - Generate an Address


## Generate a New Mnemonic 

Lets start simple. Here were just going to generate a new mnemonic.

In [ ]:
using CardanoSharp.Wallet;
using CardanoSharp.Wallet.Enums;
using CardanoSharp.Wallet.Models.Keys;
using CardanoSharp.Wallet.Extensions;

IKeyService keyService = new KeyService();
Mnemonic mnemonic = keyService.Generate(24, WordLists.English);

Console.WriteLine($"Words: {mnemonic.Words}");
Console.WriteLine($"Entropy: {mnemonic.Entropy.ToStringHex()}");

Words: blush apart receive trim color wrong ask adapt manage vintage oak earn grunt almost float jump blade argue eternal voyage trim off enhance vibrant


Entropy: 18814ace7452dbfd43501886fe865f22a6720d964bc7170171367b0e8b3292a7


## Restoring a Mnemonic

Now that we have our mnemonic, lets pretend we are loading it into a new wallet.

Here we will take our words and "Restore" them. This will produce the same object in the end.

In [ ]:
Mnemonic restoredMnemonic = keyService.Restore(mnemonic.Words, WordLists.English);

Console.WriteLine($"Words are the same: {mnemonic.Words == restoredMnemonic.Words}");
Console.WriteLine($"Entropy are the same: {mnemonic.Entropy.ToStringHex() == restoredMnemonic.Entropy.ToStringHex()}");

Words are the same: True


Entropy are the same: True


## Get the Root Key for the Mnemonic

Generating the Root Key is the first step into our Hierarchical Deterministic(HD) Wallet. This key will let us derive any key we want and we will be able to do it over and over again... Deterministic. 

As users of wallets, we are used to seeing the famous Mnemonic Words, but we also see above that there is something called Entropy. Entropy is the bytes version of your words, and this is the piece that we use to generate our Root Key. So just note that what is really going on here is Words -> Entropy -> Root Key.

In [ ]:
using CardanoSharp.Wallet.Extensions.Models;
using CardanoSharp.Wallet.Encoding;

PrivateKey rootKey = mnemonic.GetRootKey();
string rootEncoded = Bech32.Encode(rootKey.Key, "root_xsk");
Console.WriteLine($"Bech32 Encoded Root Private Key: {rootEncoded}");

Bech32 Encoded Root Private Key: root_xsk1aqymn5405csd9qmr0gsd30z236punx3hjcega6dfa3f4fn0judpnel396hpnrvrdup95xy5y2l55k83ze76fsrdhtkwe07h4dpj2uhg225705


A few points about this root key before we move on to deriving various Private and Public Keys.
 -  This is the root's private key, but the private key is more specifically an extended private key. This is defined in BIP 32.
    -  Learn more about BIP 32: [Click Here](https://en.bitcoin.it/wiki/BIP_0032)
 -  When encoding, we are following the CIP 005 prefix rules. This is why we put `root_xsk`. 
    -  Learn more about CIP 0005: [Click Here](https://github.com/cardano-foundation/CIPs/blob/master/CIP-0005/CIP-0005.md)


## Derive Private and Public Keys

Now that we can Create/Retore Mnemonics and get a Root Key, it's time to start deriving some Private and Public keys. This where we dive into the Hierarchical piece of the HD wallet. 

In [ ]:
IAddressService addressService = new AddressService();

// This path will give us our Stake Key on index 0
var stakePath = $"m/1852'/1815'/0'/2/0";
// The stakePrv is another Tuple with the Private Key and Chain Code
var stakePrv = rootKey.Derive(stakePath);
// Get the Public Key from the Stake Private Key
var stakePub = stakePrv.GetPublicKey(false); 

for(var i = 0; i < 20; i++){
    // This path will give us our Payment Key on index 0
    string paymentPath = $"m/1852'/1815'/0'/0/{i}";
    // The paymentPrv is another Tuple with the Private Key and Chain Code
    PrivateKey paymentPrv = rootKey.Derive(paymentPath); //skey
    // Get the Public Key from the Payment Private Key
    PublicKey paymentPub = paymentPrv.GetPublicKey(false); //vkey

    var baseAddr = addressService.GetAddress(
        paymentPub, 
        stakePub, 
        NetworkType.Testnet, 
        AddressType.Base);
    Console.WriteLine($"Delegation Address: {baseAddr}");
}


Delegation Address: addr_test1qq7nwxssetndd4t4z77cag8san6f9p5nxf5xajdaz3fn4tntpkvehjxmu96w84pjyzqc7ncmvessxceyhudfu5474gusugf7qr


Delegation Address: addr_test1qpwl4m2f3xcm9cansv5xk6hdvx8m2hwq4hmk49nvekn0stmtpkvehjxmu96w84pjyzqc7ncmvessxceyhudfu5474gusdj3t5w


Delegation Address: addr_test1qzlh7xy57ecg0gexa0gnscg56m364q7hs3vw3xg32avas0mtpkvehjxmu96w84pjyzqc7ncmvessxceyhudfu5474gus5mtxr8


Delegation Address: addr_test1qplhj3tfsw0syye99lrrctym9gzwnx6e473gexvuf5p4qartpkvehjxmu96w84pjyzqc7ncmvessxceyhudfu5474gusj779v5


Delegation Address: addr_test1qpgyv8y0wrxan9w57t9agvlhuw00aee7qqmk7hl6zh98jzntpkvehjxmu96w84pjyzqc7ncmvessxceyhudfu5474gusps0q2y


...

Delegation Address: addr_test1qzgr45myc6qqujt2yvvqgauc0f0wt0q9und0mk0yaup2qfntpkvehjxmu96w84pjyzqc7ncmvessxceyhudfu5474gusypvfsz


Delegation Address: addr_test1qrvqw58x4p236nw6n7jcs82u6ysj5j2q4ym2m85he5g5y6mtpkvehjxmu96w84pjyzqc7ncmvessxceyhudfu5474gusex2neu


Delegation Address: addr_test1qzrq959mmspy8v7wgkucgyz8hxcjkwzwm0h2gpdw3rdzlkrtpkvehjxmu96w84pjyzqc7ncmvessxceyhudfu5474gusvc5rkv


Delegation Address: addr_test1qqu26yt2r9gcrh3xnng9jjwhgqzcx8nd0hre3d8rp07ezzmtpkvehjxmu96w84pjyzqc7ncmvessxceyhudfu5474gusm0djdz


Delegation Address: addr_test1qz72dc9aauc94ygjkezucr7t598sg2grhelujgct3365j0mtpkvehjxmu96w84pjyzqc7ncmvessxceyhudfu5474gus029dp8
